In [2]:
import sqlite3
from pathlib import Path
import pandas as pd
import yfinance as yf
from datetime import datetime
import os

In [3]:
DB_PATH = Path(r"C:\Users\alexd\Documents\TFG_inversion_bolsa\data\ibex35.db")
DB_PATH.parent.mkdir(exist_ok=True)


In [4]:
def get_connection():
    return sqlite3.connect(DB_PATH)


In [5]:
def init_db():
    with get_connection() as conn:
        conn.execute("""
            CREATE TABLE IF NOT EXISTS ohlcv (
                ticker TEXT NOT NULL,
                date DATE NOT NULL,
                open REAL,
                high REAL,
                low REAL,
                close REAL,
                volume REAL,
                PRIMARY KEY (ticker, date)
            );
        """)
        conn.execute(
            "CREATE INDEX IF NOT EXISTS idx_ohlcv_date ON ohlcv(date);"
        )

init_db()


In [6]:
def new_download_ohlcv(ticker: str):
    tk = yf.Ticker(ticker)
    df = tk.history(period="5y", interval="1d", auto_adjust=False)

    if df.empty:
        return df

    df = df.reset_index()
    df["Date"] = pd.to_datetime(df["Date"]).dt.date

    df = df.rename(columns={
        "Date": "date",
        "Open": "open",
        "High": "high",
        "Low": "low",
        "Close": "close",
        "Volume": "volume",
    })

    df["ticker"] = ticker

    return df[["ticker", "date", "open", "high", "low", "close", "volume"]]


In [7]:
def append_ohlcv(df: pd.DataFrame):
    if df.empty:
        return 0

    with get_connection() as conn:
        df.to_sql(
            "ohlcv",
            conn,
            if_exists="append",
            index=False
        )
    return len(df)


In [7]:
from utils import get_ibex_tickers

tickers = get_ibex_tickers()

In [9]:
total_rows = 0

for t in tickers:
    print(f"Downloading {t}...")
    try:
        df = new_download_ohlcv(t)
        n = append_ohlcv(df)
        total_rows += n
        print(f"  inserted {n} rows")
    except Exception as e:
        print(f"  failed: {e}")

print(f"\nTotal rows inserted: {total_rows}")


  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1154 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 427 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows
  inserted 1280 rows

Total rows inserted: 37421
